In [ ]:
!pip install langchain langchain-community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.0 MB/s eta 0:00:00


In [ ]:
import json
import itertools
from datetime import datetime, timedelta
from typing import List, Dict, Any, Tuple
from dataclasses import dataclass
import statistics

# LangChain and DeepSeek imports
# Using OpenAI-compatible API for DeepSeek
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.agents import create_react_agent, AgentExecutor, Tool

In [ ]:
@dataclass
class Monument:
    day_id: str
    name: str
    lat: float
    lon: float
    outdoor: bool
    hours_needed: int
    days: List[Dict]

class MonumentScheduleOptimizer:
    def __init__(self, deepseek_api_key: str):
        """Initialize the optimizer with DeepSeek API key"""
        # DeepSeek uses OpenAI-compatible API
        self.llm = ChatOpenAI(
            base_url="https://api.deepseek.com",
            api_key=deepseek_api_key,
            model="deepseek-chat",
            temperature=0.1,
            max_tokens=2000
        )
        self.monuments = []

    def load_monuments(self, monuments_data: List[Dict]) -> None:
        """Load monument data into the optimizer"""
        self.monuments = []
        for monument_data in monuments_data:
            monument = Monument(
                day_id=monument_data['day_id'],
                name=monument_data['name'],
                lat=monument_data['lat'],
                lon=monument_data['lon'],
                outdoor=monument_data['outdoor'],
                hours_needed=monument_data['hours_needed'],
                days=monument_data['days']
            )
            self.monuments.append(monument)

    def get_date_range(self, start_date: str, end_date: str) -> List[str]:
        """Generate list of dates between start and end date"""
        start = datetime.strptime(start_date, '%Y-%m-%d')
        end = datetime.strptime(end_date, '%Y-%m-%d')
        dates = []
        current = start
        while current <= end:
            dates.append(current.strftime('%Y-%m-%d'))
            current += timedelta(days=1)
        return dates

    def calculate_day_score(self, day_id: str, target_date: str) -> float:
        """Calculate optimization score for assigning a day_id to a specific date"""
        monuments_for_day = [m for m in self.monuments if m.day_id == day_id]
        total_score = 0
        monument_count = 0

        for monument in monuments_for_day:
            # Find the matching day data for the target date
            day_data = None
            for day in monument.days:
                if day['date'] == target_date:
                    day_data = day
                    break

            if not day_data:
                continue

            monument_count += 1

            # Temperature penalty for outdoor monuments
            temp_penalty = 0
            if monument.outdoor:
                avg_temp = statistics.mean(day_data['temperatures'])
                # Penalty increases exponentially with temperature above 30°C
                if avg_temp > 30:
                    temp_penalty = (avg_temp - 30) ** 2

            # Crowd penalty based on day_raw values
            crowd_penalty = statistics.mean([x for x in day_data['day_raw'] if x > 0])

            # Combined score (lower is better)
            monument_score = temp_penalty + (crowd_penalty / 10)
            total_score += monument_score

        return total_score / monument_count if monument_count > 0 else float('inf')

    def generate_all_assignments(self, day_ids: List[str], dates: List[str]) -> List[Dict[str, str]]:
        """Generate all possible day-to-date assignments"""
        if len(day_ids) != len(dates):
            raise ValueError("Number of day_ids must equal number of dates")

        assignments = []
        for perm in itertools.permutations(dates):
            assignment = dict(zip(day_ids, perm))
            assignments.append(assignment)

        return assignments

    def evaluate_assignment(self, assignment: Dict[str, str]) -> float:
        """Evaluate the quality of a day-to-date assignment"""
        total_score = 0
        for day_id, date in assignment.items():
            total_score += self.calculate_day_score(day_id, date)
        return total_score

    def find_optimal_assignment(self, start_date: str, end_date: str) -> Dict[str, str]:
        """Find the optimal assignment of days to dates"""
        dates = self.get_date_range(start_date, end_date)
        day_ids = list(set([m.day_id for m in self.monuments]))

        if len(day_ids) != len(dates):
            raise ValueError(f"Number of unique day_ids ({len(day_ids)}) must equal number of dates ({len(dates)})")

        # Generate all possible assignments
        assignments = self.generate_all_assignments(day_ids, dates)

        # Find the assignment with the lowest score (best)
        best_assignment = None
        best_score = float('inf')

        for assignment in assignments:
            score = self.evaluate_assignment(assignment)
            if score < best_score:
                best_score = score
                best_assignment = assignment

        return best_assignment, best_score

class MonumentScheduleAgent:
    def __init__(self, deepseek_api_key: str):
        """Initialize the LangChain agent"""
        self.optimizer = MonumentScheduleOptimizer(deepseek_api_key)
        # DeepSeek uses OpenAI-compatible API
        self.llm = ChatOpenAI(
            base_url="https://api.deepseek.com",
            api_key=deepseek_api_key,
            model="deepseek-chat",
            temperature=0.1,
            max_tokens=1500
        )

        # Create tools for the agent
        self.tools = [
            Tool(
                name="optimize_schedule",
                description="Optimize monument visit schedule by assigning day_ids to dates. Input: start_date, end_date",
                func=self._optimize_schedule_tool
            ),
            Tool(
                name="analyze_conditions",
                description="Analyze weather and crowd conditions for specific dates. Input: date",
                func=self._analyze_conditions_tool
            ),
            Tool(
                name="get_assignment_details",
                description="Get detailed information about the optimal assignment",
                func=self._get_assignment_details_tool
            )
        ]

        # Create the agent
        self.agent_prompt = PromptTemplate(
            input_variables=["tools", "tool_names", "input", "agent_scratchpad"],
            template="""You are a monument visit scheduling expert. Your job is to optimally assign grouped monument days to specific dates.

You have access to the following tools:
{tools}

Tool names: {tool_names}

Your optimization goals (in order of priority):
1. Minimize exposure to high temperatures (>30°C) for OUTDOOR monuments
2. Avoid high crowd density periods (>70%) for all monuments
3. Provide clear, actionable recommendations

IMPORTANT INSTRUCTIONS:
- Always use the optimize_schedule tool FIRST to get the mathematically optimal assignment
- Use get_assignment_details to understand WHY the assignment is optimal
- Trust the optimization algorithm - it considers all factors simultaneously
- Only suggest manual overrides if there are compelling reasons with specific data
- Always provide the COMPLETE monument-to-date mapping in your final answer

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer should include:
1. Complete assignment (Day X -> Date, listing all monuments)
2. Optimization score and what it means
3. Key reasoning for the assignment
4. Any important warnings or recommendations

Question: {input}
{agent_scratchpad}"""
        )

        self.agent = create_react_agent(self.llm, self.tools, self.agent_prompt)
        self.agent_executor = AgentExecutor(agent=self.agent, tools=self.tools, verbose=True, handle_parsing_errors=True)

    def load_monuments(self, monuments_data: List[Dict]) -> None:
        """Load monument data"""
        self.optimizer.load_monuments(monuments_data)
        self.monuments_data = monuments_data

    def _optimize_schedule_tool(self, input_str: str) -> str:
        """Tool function for schedule optimization"""
        try:
            # Handle different input formats
            if '=' in input_str:
                # Extract dates from "start_date=X, end_date=Y" format
                parts = input_str.replace('start_date=', '').replace('end_date=', '').split(',')
            else:
                parts = input_str.split(',')

            start_date = parts[0].strip()
            end_date = parts[1].strip()

            assignment, score = self.optimizer.find_optimal_assignment(start_date, end_date)

            # Provide more detailed explanation
            result = [f"Optimal assignment found with score {score:.2f} (lower is better)"]
            result.append("Assignment details:")

            for day_id, date in assignment.items():
                monuments_for_day = [m.name for m in self.optimizer.monuments if m.day_id == day_id]
                result.append(f"  Day {day_id} -> {date}: {', '.join(monuments_for_day)}")

            result.append(f"\nJSON format: {json.dumps(assignment, indent=2)}")
            return "\n".join(result)

        except Exception as e:
            return f"Error optimizing schedule: {str(e)}"

    def _analyze_conditions_tool(self, date: str) -> str:
        """Tool function for analyzing conditions on a specific date"""
        try:
            date = date.strip()
            analysis = []

            for monument in self.optimizer.monuments:
                day_data = None
                for day in monument.days:
                    if day['date'] == date:
                        day_data = day
                        break

                if day_data:
                    avg_temp = statistics.mean(day_data['temperatures'])
                    max_crowd = max(day_data['day_raw'])
                    avg_crowd = statistics.mean([x for x in day_data['day_raw'] if x > 0])

                    analysis.append(f"{monument.name}: Avg temp {avg_temp:.1f}°C, Max crowd {max_crowd}%, Outdoor: {monument.outdoor}")

            return f"Conditions for {date}:\n" + "\n".join(analysis)
        except Exception as e:
            return f"Error analyzing conditions: {str(e)}"

    def _get_assignment_details_tool(self, input_str: str) -> str:
        """Tool function for getting detailed assignment information"""
        try:
            # Handle different input formats
            if ',' in input_str:
                parts = input_str.split(',')
                start_date = parts[0].strip()
                end_date = parts[1].strip()
            else:
                # If only one parameter provided, assume it's a request for the last optimization
                # For now, return a helpful message
                return "Please provide both start_date and end_date separated by comma, e.g., '2025-08-04, 2025-08-05'"

            assignment, score = self.optimizer.find_optimal_assignment(start_date, end_date)

            details = [f"DETAILED ASSIGNMENT ANALYSIS"]
            details.append(f"Optimization Score: {score:.2f} (lower is better)")
            details.append(f"Score components: temperature penalties + crowd penalties")
            details.append("")

            for day_id, date in assignment.items():
                monuments_for_day = [m for m in self.optimizer.monuments if m.day_id == day_id]
                details.append(f"📅 Day {day_id} assigned to {date}:")

                total_day_score = 0
                for monument in monuments_for_day:
                    day_data = None
                    for day in monument.days:
                        if day['date'] == date:
                            day_data = day
                            break

                    if day_data:
                        avg_temp = statistics.mean(day_data['temperatures'])
                        max_temp = max(day_data['temperatures'])
                        avg_crowd = statistics.mean([x for x in day_data['day_raw'] if x > 0])
                        max_crowd = max(day_data['day_raw'])

                        # Calculate individual score
                        temp_penalty = 0
                        if monument.outdoor and avg_temp > 30:
                            temp_penalty = (avg_temp - 30) ** 2
                        crowd_penalty = avg_crowd / 10
                        monument_score = temp_penalty + crowd_penalty
                        total_day_score += monument_score

                        outdoor_indicator = "🌞 OUTDOOR" if monument.outdoor else "🏛️ INDOOR"
                        details.append(f"  {outdoor_indicator} {monument.name}")
                        details.append(f"    🌡️ Temperature: {avg_temp:.1f}°C avg, {max_temp:.1f}°C max")
                        details.append(f"    👥 Crowd: {avg_crowd:.0f}% avg, {max_crowd}% max")
                        details.append(f"    📊 Score: {monument_score:.2f}")

                        # Add warnings
                        if monument.outdoor and avg_temp > 32:
                            details.append("    ⚠️ HIGH TEMPERATURE for outdoor visit!")
                        if avg_crowd > 70:
                            details.append("    ⚠️ HIGH CROWD DENSITY expected!")
                        details.append("")

                details.append(f"  Total day score: {total_day_score:.2f}")
                details.append("")

            return "\n".join(details)

        except Exception as e:
            return f"Error getting assignment details: {str(e)}"

    def optimize_schedule(self, start_date: str, end_date: str) -> str:
        """Main method to optimize monument schedule"""
        query = f"Optimize the monument visit schedule from {start_date} to {end_date}. Consider weather conditions and crowd levels. Provide detailed reasoning for the optimal assignment."

        try:
            result = self.agent_executor.invoke({"input": query})
            return result["output"]
        except Exception as e:
            # Fallback to direct optimization if agent fails
            assignment, score = self.optimizer.find_optimal_assignment(start_date, end_date)
            return f"Optimal assignment (fallback): {json.dumps(assignment, indent=2)}\nScore: {score:.2f}"


In [ ]:
monuments_data = [
  {
    "day_id": "A",
    "name": "Cairo Tower",
    "lat": 30.0459751,
    "lon": 31.2242988,
    "outdoor": False,
    "hours_needed": 3,
    "days": [
      {
        "day_text": "Monday",
        "venue_open": 9,
        "venue_closed": 1,
        "day_raw": [
          0,
          0,
          0,
          20,
          25,
          30,
          35,
          35,
          40,
          45,
          55,
          65,
          75,
          80,
          80,
          70,
          60,
          45,
          25,
          0,
          0,
          0,
          0,
          0
        ],
        "lat": 30.0459751,
        "long": 31.2242988,
        "date": "2025-08-04",
        "temperatures": [
          26.3,
          25.7,
          25.1,
          24.5,
          23.9,
          23.5,
          23.2,
          23.3,
          24.3,
          26.1,
          27.7,
          29.4,
          30.9,
          32.1,
          33.0,
          33.6,
          34.2,
          34.0,
          33.8,
          33.2,
          32.0,
          30.2,
          28.5,
          27.4
        ]
      },
      {
        "day_text": "Tuesday",
        "venue_open": 9,
        "venue_closed": 1,
        "day_raw": [
          0,
          0,
          0,
          20,
          25,
          30,
          35,
          40,
          40,
          45,
          50,
          60,
          70,
          80,
          80,
          75,
          65,
          50,
          30,
          0,
          0,
          0,
          0,
          0
        ],
        "lat": 30.0459751,
        "long": 31.2242988,
        "date": "2025-08-05",
        "temperatures": [
          26.6,
          25.8,
          25.1,
          24.4,
          23.9,
          23.5,
          23.3,
          23.2,
          24.3,
          26.0,
          27.4,
          29.0,
          30.2,
          31.6,
          32.7,
          33.5,
          34.0,
          34.2,
          34.1,
          33.5,
          31.9,
          30.0,
          28.5,
          27.1
        ]
      },
      {
        "day_text": "Sunday",
        "venue_open": 9,
        "venue_closed": 1,
        "day_raw": [
          0,
          0,
          0,
          25,
          30,
          40,
          40,
          45,
          45,
          50,
          60,
          70,
          80,
          90,
          90,
          90,
          75,
          55,
          35,
          0,
          0,
          0,
          0,
          0
        ],
        "lat": 30.0459751,
        "long": 31.2242988,
        "date": "2025-08-03",
        "temperatures": [
          26.6,
          26.2,
          25.9,
          25.5,
          24.9,
          24.4,
          23.9,
          23.8,
          24.8,
          26.8,
          28.5,
          30.2,
          31.6,
          33.0,
          34.0,
          34.4,
          34.5,
          34.3,
          33.9,
          32.9,
          31.3,
          30.3,
          28.0,
          27.1
        ]
      }
    ]
  },
  {
    "day_id": "A",
    "name": "The Egyptian Museum in Cairo",
    "lat": 30.0483167,
    "lon": 31.2336674,
    "outdoor": False,
    "hours_needed": 3,
    "days": [
      {
        "day_text": "Monday",
        "venue_open": 9,
        "venue_closed": 17,
        "day_raw": [
          0,
          0,
          0,
          50,
          65,
          70,
          70,
          70,
          75,
          70,
          60,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0
        ],
        "lat": 30.0483167,
        "long": 31.2336674,
        "date": "2025-08-04",
        "temperatures": [
          26.3,
          25.7,
          25.1,
          24.5,
          23.9,
          23.5,
          23.2,
          23.3,
          24.3,
          26.1,
          27.7,
          29.4,
          30.9,
          32.1,
          33.0,
          33.6,
          34.2,
          34.0,
          33.8,
          33.2,
          32.0,
          30.2,
          28.5,
          27.4
        ]
      },
      {
        "day_text": "Tuesday",
        "venue_open": 9,
        "venue_closed": 17,
        "day_raw": [
          0,
          0,
          0,
          60,
          70,
          75,
          75,
          80,
          85,
          80,
          70,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0
        ],
        "lat": 30.0483167,
        "long": 31.2336674,
        "date": "2025-08-05",
        "temperatures": [
          26.6,
          25.8,
          25.1,
          24.4,
          23.9,
          23.5,
          23.3,
          23.2,
          24.3,
          26.0,
          27.4,
          29.0,
          30.2,
          31.6,
          32.7,
          33.5,
          34.0,
          34.2,
          34.1,
          33.5,
          31.9,
          30.0,
          28.5,
          27.1
        ]
      },
      {
        "day_text": "Sunday",
        "venue_open": 9,
        "venue_closed": 17,
        "day_raw": [
          0,
          0,
          0,
          65,
          85,
          90,
          95,
          100,
          100,
          90,
          75,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0
        ],
        "lat": 30.0483167,
        "long": 31.2336674,
        "date": "2025-08-03",
        "temperatures": [
          26.6,
          26.2,
          25.9,
          25.5,
          24.9,
          24.4,
          23.9,
          23.8,
          24.8,
          26.8,
          28.5,
          30.2,
          31.6,
          33.0,
          34.0,
          34.4,
          34.5,
          34.3,
          33.9,
          32.9,
          31.3,
          30.3,
          28.0,
          27.1
        ]
      }
    ]
  }
  ,{
    "day_id": "B",
    "name": "Grand Egyptian Museum (GEM)",
    "lat": 29.9943546,
    "lon": 31.1192991,
    "outdoor": False,
    "hours_needed": 3,
    "days": [
      {
        "day_text": "Monday",
        "venue_open": 8,
        "venue_closed": 19,
        "day_raw": [
          0,
          0,
          10,
          15,
          25,
          40,
          50,
          60,
          65,
          65,
          55,
          40,
          30,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0
        ],
        "lat": 29.9943546,
        "long": 31.1192991,
        "date": "2025-08-04",
        "temperatures": [
          26.1,
          25.4,
          24.8,
          24.2,
          23.7,
          23.3,
          23.1,
          23.0,
          23.9,
          25.5,
          27.2,
          28.8,
          30.3,
          31.7,
          32.6,
          33.4,
          33.8,
          33.8,
          33.5,
          32.9,
          31.7,
          29.7,
          28.2,
          27.1
        ]
      },
      {
        "day_text": "Tuesday",
        "venue_open": 8,
        "venue_closed": 19,
        "day_raw": [
          0,
          0,
          10,
          20,
          30,
          40,
          55,
          65,
          70,
          65,
          55,
          45,
          30,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0
        ],
        "lat": 29.9943546,
        "long": 31.1192991,
        "date": "2025-08-05",
        "temperatures": [
          26.2,
          25.4,
          24.7,
          24.0,
          23.5,
          23.2,
          22.8,
          22.9,
          23.9,
          25.5,
          27.2,
          28.5,
          29.9,
          31.3,
          32.5,
          33.2,
          33.9,
          34.1,
          33.8,
          33.1,
          31.4,
          29.6,
          28.0,
          26.6
        ]
      },
      {
        "day_text": "Sunday",
        "venue_open": 8,
        "venue_closed": 19,
        "day_raw": [
          0,
          0,
          10,
          20,
          30,
          45,
          55,
          70,
          75,
          75,
          65,
          50,
          35,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0
        ],
        "lat": 29.9943546,
        "long": 31.1192991,
        "date": "2025-08-03",
        "temperatures": [
          26.5,
          25.9,
          25.5,
          25.0,
          24.5,
          24.1,
          23.7,
          23.6,
          24.5,
          26.4,
          28.1,
          29.5,
          31.0,
          32.4,
          33.3,
          33.8,
          33.9,
          33.8,
          33.5,
          32.7,
          31.1,
          29.9,
          27.8,
          26.9
        ]
      }
    ]
  },
  {
    "day_id": "B",
    "name": "Giza Necropolis",
    "lat": 29.9772962,
    "lon": 31.1324955,
    "outdoor": True,
    "hours_needed": 3,
    "days": [
      {
        "day_text": "Monday",
        "venue_open": 6,
        "venue_closed": 16,
        "day_raw": [
          5,
          15,
          25,
          40,
          55,
          65,
          65,
          65,
          65,
          60,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0
        ],
        "lat": 29.9772962,
        "long": 31.1324955,
        "date": "2025-08-04",
        "temperatures": [
          25.8,
          25.1,
          24.5,
          23.9,
          23.4,
          23.0,
          22.8,
          22.7,
          23.6,
          25.2,
          26.9,
          28.5,
          30.0,
          31.4,
          32.3,
          33.1,
          33.5,
          33.5,
          33.2,
          32.6,
          31.4,
          29.4,
          27.9,
          26.8
        ]
      },
      {
        "day_text": "Tuesday",
        "venue_open": 6,
        "venue_closed": 16,
        "day_raw": [
          5,
          15,
          30,
          50,
          70,
          75,
          75,
          75,
          70,
          65,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0
        ],
        "lat": 29.9772962,
        "long": 31.1324955,
        "date": "2025-08-05",
        "temperatures": [
          25.9,
          25.1,
          24.4,
          23.7,
          23.2,
          22.9,
          22.5,
          22.6,
          23.6,
          25.2,
          26.9,
          28.2,
          29.6,
          31.0,
          32.2,
          32.9,
          33.6,
          33.8,
          33.5,
          32.8,
          31.1,
          29.3,
          27.7,
          26.3
        ]
      },
      {
        "day_text": "Sunday",
        "venue_open": 6,
        "venue_closed": 16,
        "day_raw": [
          5,
          15,
          30,
          45,
          65,
          80,
          80,
          80,
          75,
          70,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0
        ],
        "lat": 29.9772962,
        "long": 31.1324955,
        "date": "2025-08-03",
        "temperatures": [
          26.2,
          25.6,
          25.2,
          24.7,
          24.2,
          23.8,
          23.4,
          23.3,
          24.2,
          26.1,
          27.8,
          29.2,
          30.7,
          32.1,
          33.0,
          33.5,
          33.6,
          33.5,
          33.2,
          32.4,
          30.8,
          29.6,
          27.5,
          26.6
        ]
      }
    ]
  }
]

In [ ]:
# Initialize agent
api_key = "sk-your_deepseek_api_key"  # Replace with actual API key
agent = MonumentScheduleAgent(api_key)
agent.load_monuments(monuments_data)

# Test optimization
result = agent.optimize_schedule("2025-08-04", "2025-08-05")
print("Agent Result:")
print(result)



> Entering new AgentExecutor chain...
Thought: To optimize the monument visit schedule, I should first use the optimize_schedule tool to get the mathematically optimal assignment based on the given dates and optimization goals.

Action: optimize_schedule
Action Input: start_date=2025-08-04, end_date=2025-08-05
Optimal assignment found with score 10.57 (lower is better)
Assignment details:
  Day A -> 2025-08-05: Cairo Tower, The Egyptian Museum in Cairo
  Day B -> 2025-08-04: Grand Egyptian Museum (GEM), Giza Necropolis

JSON format: {
  "A": "2025-08-05",
  "B": "2025-08-04"
}Now that I have the optimal assignment, I should use get_assignment_details to understand why this particular assignment was chosen by the algorithm.

Action: get_assignment_details
Action Input: start_date=2025-08-04, end_date=2025-08-05
Error getting assignment details: time data 'start_date=2025-08-04' does not match format '%Y-%m-%d'It seems there was an error in the input format for the get_assignment_detai